In [ ]:
# pip install mysql-connector-python

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
    --------------------------------------- 0.3/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.5 MB 2.1 MB/s eta 0:00:08
   - -------------------------------------- 0.8/16.5 MB 2.1 MB/s eta 0:00:08
   ----- ---------------------------------- 2.1/16.5 MB 2.8 MB/s eta 0:00:06
   ------ --------------------------------- 2.6/16.5 MB 2.8 MB/s eta 0:00:05
   -------- ------------------------------- 3.4/16.5 MB 3.1 MB/s eta 0:00:05
   --------- ------------------------------ 3.9/16.5 MB 3.0 MB/s eta 0:00:05
   ---------- ----------------------------- 4.5/16.5 MB 3.0 MB/s eta 0:00:05
   ---------- ----------------------------- 4.5/16.5 MB 3.0 MB/s eta 0:00:05
   ----------- ---------------------------- 4.7/16.5 MB 2.5 MB/s eta 0:00:05
   ------------ --------------------------- 5.0/16.5 MB 2.3 MB/s eta 0:00:06
   ------------ ----

In [5]:
import sqlite3
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

In [6]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS energy_data (
        MeterID TEXT,
        Zone TEXT,
        ConsumerType TEXT,
        Date TEXT,
        EnergyConsumed_kWh REAL,
        PeakUsage_kWh REAL,
        OutageMinutes INTEGER,
        MeterStatus TEXT,
        TariffRate REAL
    )
''')


In [8]:
with open('SmartCityEnergy_Synthetic.csv','r') as file:
    next(file)  # Skip header row
    for line in file:
        fields = line.strip().split(',')
        cursor.execute('''
            INSERT INTO energy_data (
                MeterID, Zone, ConsumerType, Date,
                EnergyConsumed_kWh, PeakUsage_kWh,
                OutageMinutes, MeterStatus, TariffRate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', fields)

conn.commit()

In [12]:
cursor.execute('SELECT * FROM energy_data LIMIT 5')
rows = cursor.fetchall()
for row in rows:
    print(row)

('MTR0001', 'East', 'Residential', '11/24/2025', 3108.34, 301.44, 145, 'Faulty', 7.75)
('MTR0002', 'West', 'Residential', '2/27/2025', 4351.08, 261.91, 154, 'Active', 7.42)
('MTR0003', 'Central', 'Industrial', '1/13/2025', 2874.52, 118.59, 54, 'Active', 4.89)
('MTR0004', 'West', 'Commercial', '5/21/2025', 200.42, 12.47, 119, 'Active', 4.74)
('MTR0005', 'West', 'Residential', '5/6/2025', 4658.2, 104.94, 74, 'Active', 9.59)


## Q-1 🔌 Total and average daily energy consumption by zone.

In [14]:
cursor.execute('SELECT zone, AVG(EnergyConsumed_kWh) as AvgEnergy, SUM(EnergyConsumed_kWh) as TotalEnergy FROM energy_data GROUP BY zone')
results = cursor.fetchall()
for result in results:
    print(result)

('Central', 2487.7085263157896, 945329.24)
('East', 2520.5476699029127, 1038465.64)
('North', 2503.9317142857144, 1051651.32)
('South', 2628.4816842105265, 998823.04)
('West', 2413.920196078431, 984879.44)


## Q-2 ⚡ Identify top 5 highest energy-consuming consumers by type.

In [16]:
cursor.execute('SELECT MeterID, ConsumerType, EnergyConsumed_kWh FROM (SELECT MeterID, ConsumerType, EnergyConsumed_kWh, RANK() OVER (PARTITION BY ConsumerType ORDER BY EnergyConsumed_kWh DESC) as rank FROM energy_data) WHERE rank <= 5 ORDER BY ConsumerType, EnergyConsumed_kWh DESC')
top_consumers = cursor.fetchall()
for consumer in top_consumers:
    print(consumer)

('MTR0014', 'Commercial', 4997.1)
('MTR0014', 'Commercial', 4997.1)
('MTR0277', 'Commercial', 4983.65)
('MTR0277', 'Commercial', 4983.65)
('MTR0462', 'Commercial', 4981.85)
('MTR0462', 'Commercial', 4981.85)
('MTR0864', 'Industrial', 4964.44)
('MTR0864', 'Industrial', 4964.44)
('MTR0141', 'Industrial', 4956.72)
('MTR0141', 'Industrial', 4956.72)
('MTR0134', 'Industrial', 4939.54)
('MTR0134', 'Industrial', 4939.54)
('MTR0504', 'Residential', 4991.82)
('MTR0504', 'Residential', 4991.82)
('MTR0265', 'Residential', 4974.59)
('MTR0265', 'Residential', 4974.59)
('MTR0166', 'Residential', 4970.99)
('MTR0166', 'Residential', 4970.99)


## Q-3 📈 Monthly trend of consumption across zones.

In [25]:
cursor.execute("SELECT Zone, strftime('%m', substr(Date, 7, 4) || '-' || substr(Date, 1, 2) || '-' || substr(Date, 4, 2)) as Month, SUM(EnergyConsumed_kWh) as MonthlyConsumption FROM energy_data GROUP BY Zone, Month")
monthly_trends = cursor.fetchall()
for trend in monthly_trends:
    print(trend)


('Central', None, 767646.34)
('Central', '10', 63578.88)
('Central', '11', 26095.739999999998)
('Central', '12', 88008.28)
('East', None, 854625.04)
('East', '10', 76170.1)
('East', '11', 59477.38)
('East', '12', 48193.12)
('North', None, 893191.44)
('North', '10', 61838.92)
('North', '11', 46846.14)
('North', '12', 49774.82)
('South', None, 831698.9)
('South', '10', 60130.38)
('South', '11', 61687.46)
('South', '12', 45306.3)
('West', None, 823739.56)
('West', '10', 56705.48)
('West', '11', 52819.36)
('West', '12', 51615.04)


## Q-4  Calculate average cost per zone (EnergyConsumed × TariffRate).

In [26]:
cursor.execute('SELECT Zone, AVG(EnergyConsumed_kwh*TariffRate) as AvgCost FROM energy_data GROUP BY Zone')
costs = cursor.fetchall()
for cost in costs:
    print(cost)

('Central', 16587.945031578947)
('East', 16571.62966747573)
('North', 16785.92351952381)
('South', 16376.02776526316)
('West', 15237.875181372548)


## Q-5 📉 List meters with highest number of faults or outages.

In [28]:
cursor.execute('SELECT MeterID, COUNT(*) as OutageCount FROM energy_data WHERE MeterStatus="Faulty"  GROUP by MeterID order by OutageCount DESC LIMIT 10')
outage_prone_meters = cursor.fetchall()
for meter in outage_prone_meters:
    print(meter)

('MTR0999', 2)
('MTR0996', 2)
('MTR0995', 2)
('MTR0994', 2)
('MTR0993', 2)
('MTR0992', 2)
('MTR0990', 2)
('MTR0989', 2)
('MTR0988', 2)
('MTR0982', 2)


## Q-6 ♻️ Determine zones with lowest energy efficiency (high usage + frequent outages).

In [32]:
cursor.execute('SELECT Zone, (SUM(EnergyConsumed_kwh)+SUM(OutageMinutes*0.1)) as TotalImpact FROM energy_data GROUP BY Zone ORDER BY TotalImpact DESC')
impactful_zones = cursor.fetchall()
for zone in impactful_zones:
    print(zone)


('North', 1055621.52)
('East', 1042063.24)
('South', 1002473.24)
('West', 988743.0399999999)
('Central', 948667.64)


## Q-7 🗓️ Detect patterns of peak usage during weekdays vs weekends.

In [ ]:
cursor.execute('SELECT ')